In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

device = "cuda" # 加载模型的设备

model = AutoModelForCausalLM.from_pretrained(
   "M:\model\Qwen1.5_0.5b_chat",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("M:\model\Qwen1.5_0.5b_chat")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

c:\Users\gu\AppData\Local\conda\conda\envs\py310\lib\site-packages\accelerate\utils\modeling.py:1365: UserWarning: Current model requires 738203136 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from langchain_community.document_loaders import PDFPlumberLoader
file_path = '1关于数字化改革总体方案的汇报（省委改革办）.pdf'
loader = PDFPlumberLoader(file_path)
data = loader.load()


In [3]:
# 文本分割
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits = text_splitter.split_documents(data)

In [4]:
# 向量化存储
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "./bge-large-zh-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bgeEmbeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\gu\AppData\Local\conda\conda\envs\py310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
# 向量数据库
from langchain.vectorstores import FAISS
vector = FAISS.from_documents(all_splits, bgeEmbeddings)


In [6]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# retriever.invoke("省委改革办数字化总体方案要求有哪些？")

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import BaseChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""仅根据所提供的上下文回答以下问题:

<context>
{context}
</context>

问题: {question}""")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
retriever_chain: RunnableSequence = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    # | StrOutputParser()
)

In [8]:
ans1 = retriever_chain.invoke("省委改革办数字化总体方案要求有哪些？")
# ans1 = retriever_chain.invoke("省委改革办数字化总体方案要求有哪些？")
print(ans1)


In [ ]:
ans2 = retriever_chain.invoke("省委改革办数字化改革总体方案的汇报中的“1 5 2”任务分别指哪些？")
print(ans2)